# MDF data analysis

In [1]:
from load_data import train, test, BIG, TRAIN, TEST
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
%matplotlib inline

TRAIN (8564, 42)
TEST  (3671, 42)
BIG  (12235, 43)


/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
from visualization_helper import display_groups, display_unique_count, display_unique_col_values, split_series_values

In [3]:
print train.shape, train.columns.values

(8564, 42) [u'libelle' u'libelle_plaquette' u'libelle_ampoule' u'libelle_flacon'
 u'libelle_tube' u'libelle_stylo' u'libelle_seringue' u'libelle_pilulier'
 u'libelle_sachet' u'libelle_comprime' u'libelle_gelule' u'libelle_film'
 u'libelle_poche' u'libelle_capsule' u'nb_plaquette' u'nb_ampoule'
 u'nb_flacon' u'nb_tube' u'nb_stylo' u'nb_seringue' u'nb_pilulier'
 u'nb_sachet' u'nb_comprime' u'nb_gelule' u'nb_film' u'nb_poche'
 u'nb_capsule' u'nb_ml' u'statut' u'etat commerc' u'agrement col'
 u'tx rembours' u'forme pharma' u'voies admin' u'statut admin'
 u'date declar annee' u'date amm annee' u'type proc' u'titulaires'
 u'substances' u'prix' 'source']


Remove some useless columns

In [4]:
train__noid_noprix = train.drop(['prix','libelle'], axis=1)
print train__noid_noprix.shape, train__noid_noprix.columns.values

(8564, 40) [u'libelle_plaquette' u'libelle_ampoule' u'libelle_flacon' u'libelle_tube'
 u'libelle_stylo' u'libelle_seringue' u'libelle_pilulier' u'libelle_sachet'
 u'libelle_comprime' u'libelle_gelule' u'libelle_film' u'libelle_poche'
 u'libelle_capsule' u'nb_plaquette' u'nb_ampoule' u'nb_flacon' u'nb_tube'
 u'nb_stylo' u'nb_seringue' u'nb_pilulier' u'nb_sachet' u'nb_comprime'
 u'nb_gelule' u'nb_film' u'nb_poche' u'nb_capsule' u'nb_ml' u'statut'
 u'etat commerc' u'agrement col' u'tx rembours' u'forme pharma'
 u'voies admin' u'statut admin' u'date declar annee' u'date amm annee'
 u'type proc' u'titulaires' u'substances' 'source']


Check for dupicate rows.

In [5]:
duplicated = train__noid_noprix.duplicated()
np.sum(duplicated)

1164

In [6]:
train__noid_noprix_unique = train__noid_noprix.drop_duplicates()
len(train__noid_noprix_unique)

7400

Lets look at the columns with only one unique value.

In [7]:
def drop_const_cols(df):
    return df.loc[:, (df != df.ix[0]).any()]

train__noid_noprix_unique_noconst = drop_const_cols(train__noid_noprix_unique)
print train__noid_noprix_unique.shape, '->', train__noid_noprix_unique_noconst.shape


(7400, 40) -> (7400, 39)


In [8]:
print "Dropped const columns : "
if len(train__noid_noprix_unique.columns) > len(train__noid_noprix_unique_noconst.columns):
    print train__noid_noprix_unique.columns.difference(train__noid_noprix_unique_noconst.columns).values


Dropped const columns : 
['source']


Columns description : 

In [9]:
type_groups_func = lambda df : df.columns.to_series().groupby(df.dtypes)
type_groups = type_groups_func(train__noid_noprix_unique_noconst)

In [10]:
display_groups(type_groups)


-- object --

	statut
	etat commerc
	agrement col
	tx rembours
	forme pharma
	voies admin
	statut admin
	type proc
	titulaires
	substances

-- int64 --

	libelle_plaquette
	libelle_ampoule
	libelle_flacon
	libelle_tube
	libelle_stylo
	libelle_seringue
	libelle_pilulier
	libelle_sachet
	libelle_comprime
	libelle_gelule
	libelle_film
	libelle_poche
	libelle_capsule
	nb_plaquette
	nb_ampoule
	nb_flacon
	nb_tube
	nb_stylo
	nb_seringue
	nb_pilulier
	nb_sachet
	nb_comprime
	nb_gelule
	nb_film
	nb_poche
	nb_capsule
	date declar annee
	date amm annee

-- float64 --

	nb_ml


Identify and separate the numeric and non numeric rows.
Report NaN values on numerical columns

In [11]:
numeric_cols = []
nonnum_cols = []
for key in type_groups.groups:
    if np.issubdtype(key, np.number):
        numeric_cols.extend(type_groups.get_group(key).values)
    else:
        nonnum_cols.extend(type_groups.get_group(key).values)
        
nb_isnan = np.sum(train__noid_noprix_unique_noconst[numeric_cols].apply(np.isnan))

In [12]:
print nb_isnan

libelle_plaquette    0
libelle_ampoule      0
libelle_flacon       0
libelle_tube         0
libelle_stylo        0
libelle_seringue     0
libelle_pilulier     0
libelle_sachet       0
libelle_comprime     0
libelle_gelule       0
libelle_film         0
libelle_poche        0
libelle_capsule      0
nb_plaquette         0
nb_ampoule           0
nb_flacon            0
nb_tube              0
nb_stylo             0
nb_seringue          0
nb_pilulier          0
nb_sachet            0
nb_comprime          0
nb_gelule            0
nb_film              0
nb_poche             0
nb_capsule           0
date declar annee    0
date amm annee       0
nb_ml                0
dtype: int64


Identify non numeric rows.

In [13]:
train__noid_noprix_unique_noconst[nonnum_cols].tail(5)

,statut,etat commerc,agrement col,tx rembours,forme pharma,voies admin,statut admin,type proc,titulaires,substances
8553,Présentation active,Déclaration de commercialisation,oui,65%,solution injectable,intraveineuse,Autorisation active,Procédure nationale,BRACCO IMAGING FRANCE,GADOTÉRIDOL
8556,Présentation active,Déclaration de commercialisation,oui,65%,solution injectable,sous-cutanée,Autorisation active,Procédure centralisée,BRISTOL MYERS SQUIBB PHARMA (GRANDE BRETAGNE),ABATACEPT
8557,Présentation active,Déclaration de commercialisation,oui,65%,gélule,orale,Autorisation active,Procédure nationale,BIOGARAN,"FLUVASTATINE, FLUVASTATINE SODIQUE"
8559,Présentation active,Déclaration d'arrêt de commercialisation,non,65%,comprimé orodispersible,orale,Autorisation active,Procédure décentralisée,RATIOPHARM (ALLEMAGNE),OLANZAPINE
8560,Présentation active,Déclaration de commercialisation,oui,65%,solution injectable,sous-cutanée,Autorisation active,Procédure centralisée,MERCK SERONO EUROPE (ROYAUME-UNI),INTERFÉRON BÊTA-1A


In [14]:
unique_col_values = display_unique_col_values(train__noid_noprix_unique_noconst[nonnum_cols])
unique_col_values

,0,1,2,3,4,5,6,7,8,9,...,1732,1733,1734,1735,1736,1737,1738,1739,1740,1741
0,statut,2,Présentation active,Présentation abrogée,,,,,,,...,,,,,,,,,,
1,etat commerc,4,Déclaration de commercialisation,Déclaration d'arrêt de commercialisation,Arrêt de commercialisation (le médicament n'a ...,Déclaration de suspension de commercialisation,,,,,...,,,,,,,,,,
2,agrement col,2,oui,non,,,,,,,...,,,,,,,,,,
3,tx rembours,5,65%,100%,15%,30%,0%,,,,...,,,,,,,,,,
4,forme pharma,190,comprimé pelliculé,comprimé à croquer,gélule à libération prolongée,granulés à libération prolongée,comprimé,capsule molle,comprimé sécable,dispositif,...,,,,,,,,,,
5,voies admin,75,orale,transdermique,inhalée,"orale,vaginale",ophtalmique,sous-cutanée,auriculaire,"sous-cutanée,voie extracorporelle autre",...,,,,,,,,,,
6,statut admin,4,Autorisation active,Autorisation abrogée,Autorisation archivée,Autorisation retirée,,,,,...,,,,,,,,,,
7,type proc,5,Procédure décentralisée,Procédure de reconnaissance mutuelle,Procédure nationale,Procédure centralisée,Autorisation d'importation parallèle,,,,...,,,,,,,,,,
8,titulaires,325,MAJORELLE LUXEMBOURG SOPARFI (LUXEMBOURG),TEVA SANTE,MYLAN SAS,SANOFI AVENTIS FRANCE,EG LABO - LABORATOIRES EUROGENERICS,EISAI (ROYAUME-UNI),SANDOZ,ARROW GENERIQUES,...,,,,,,,,,,
9,substances,1740,DÉSOGESTREL,"MONTÉLUKAST ACIDE, MONTÉLUKAST SODIQUE",CHLORHYDRATE DE DILTIAZEM,"ACIDE VALPROÏQUE, VALPROATE DE SODIUM",LÉTROZOLE,FLUTAMIDE,BEXAROTÈNE,GLIMÉPIRIDE,...,N-ACÉTYL L-CYSTÉINE,"LISINOPRIL, LISINOPRIL DIHYDRATÉ","ACIDE ALENDRONIQUE, ALENDRONATE MONOSODIQUE MO...",OXYTOCINE,"CONCENTRAT DE CHOLÉCALCIFÉROL, FORME PULVÉRULE...","MANGANÈSE (GLYCÉROPHOSPHATE DE), GLYCÉROPHOSPH...",EXÉNATIDE,"HYDROGÉNOTARTRATE DE RIVASTIGMINE, RIVASTIGMINE","CALCIUM ÉLÉMENT, CALCIUM (CARBONATE DE), CALCI...",ABATACEPT


In [25]:
index = 5
ll = unique_col_values.iloc[index][1]
res = split_series_values(unique_col_values.iloc[index][2:ll])    
df = display_unique_count(res)
print np.sum(df['Count'])
print df

201
                           Value  Count
23                 intraveineuse     32
12             intra-articulaire     17
13              intra-artérielle     15
19               intramusculaire     14
29                         orale     11
22                  intrathécale     10
35                  sous-cutanée      9
14                 intra-utérine      8
30               périarticulaire      7
34                       rectale      6
31                    péridurale      6
4                  endocanalaire      5
18              intralésionnelle      5
32                   périneurale      4
5                   endosinusale      4
9                   infiltration      4
40    voie extracorporelle autre      3
33                  périoculaire      3
39            voie buccale autre      3
8                      gingivale      3
41        voie parentérale autre      3
27                        nasale      2
37                 transdermique      2
38                      vaginale    

In [26]:
index = 8
ll = unique_col_values.iloc[index][1]
res = split_series_values(unique_col_values.iloc[index][2:ll])    
df = display_unique_count(res)
print np.sum(df['Count'])
print df

325
                                                Value  Count
301                                   TEVA (PAYS-BAS)      2
208             MERCK SHARP & DOHME (GRANDE BRETAGNE)      2
213                                       MUNDIPHARMA      1
220                                NORGINE (PAYS-BAS)      1
219                                     NORDIC PHARMA      1
218                                     NEXMED PHARMA      1
217                                            NEPALM      1
216                                         NELSONIUS      1
215                                         MYLAN SAS      1
214                                     MYLAN MEDICAL      1
0                                     ABBOTT PRODUCTS      1
221                                    NORGINE PHARMA      1
211                                  MOLTENI & ALITTI      1
210                                       MITHRIDATUM      1
209                    MERCK SHARP & DOHME (PAYS-BAS)      1
207                 

In [27]:
index = 9
ll = unique_col_values.iloc[index][1]
res = split_series_values(unique_col_values.iloc[index][2:ll])    
df = display_unique_count(res)
print np.sum(df['Count'])
print df

3800
                                        Value  Count
287                  CLAVULANATE DE POTASSIUM     33
77                   AMOXICILLINE TRIHYDRATÉE     31
463                       HYDROCHLOROTHIAZIDE     30
1508                      HYDROCHLOROTHIAZIDE     24
10                         ACIDE CLAVULANIQUE     24
162                           CALCIUM ÉLÉMENT     22
69                                 AMLODIPINE     22
268                           CHOLÉCALCIFÉROL     21
901                          ÉTHINYLESTRADIOL     18
442                       GLUCOSE MONOHYDRATÉ     18
706                               PÉRINDOPRIL     18
409                        FORME PULVÉRULENTE     18
74                       AMOXICILLINE ANHYDRE     17
47                                    ALANINE     16
141                     BÉSILATE D'AMLODIPINE     16
499                                ISOLEUCINE     16
858                                    VALINE     16
850                             TRYPTOPHA

Encode categorical features

In [16]:
df = train__noid_noprix_unique_noconst[nonnum_cols].drop(['substances', 'voies admin'], axis=1)
out_df = pd.DataFrame(columns=df.columns, index=df.index)
for c in df.columns:
    le = LabelEncoder()    
    le.fit(df[c])
    out_df[c] = le.transform(df[c])
out_df

,statut,etat commerc,agrement col,tx rembours,forme pharma,statut admin,type proc,titulaires
0,1,2,1,4,30,1,3,194
1,1,2,1,4,49,1,2,307
2,1,2,1,4,78,1,4,218
3,1,2,1,4,74,1,4,275
4,1,1,0,1,30,1,4,307
5,1,2,1,1,7,1,4,92
6,1,2,1,1,1,1,1,93
7,1,2,1,4,7,1,2,272
8,1,1,1,4,44,1,4,32
9,1,2,0,4,30,1,4,32
